<a href="https://colab.research.google.com/github/Shodef/Assited-hatching/blob/main/%E2%80%9Cpubmed_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install biopython pandas googletrans==4.0.0-rc1 python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 23.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=0b1ad85b02d1aec5e090247e986e2e2128330329bf428538352361f421f04f6e
  Stored in directory: /root/.cache/pi

In [ ]:
import pandas as pd
from Bio import Entrez
from googletrans import Translator
from docx import Document

# 设置你的email，Entrez需要用到
Entrez.email = "liuchangdong007@gmail.com"

# 使用Entrez搜索PubMed
search_results = Entrez.read(
    Entrez.esearch(
        db="pubmed", term="Assisted hatching of the embryos", retmax=500
    )
)

# 获取所有文章的ID
id_list = search_results["IdList"]

# 获取所有文章的详细信息
articles = Entrez.read(Entrez.efetch(db="pubmed", id=id_list, retmode="xml"))

article_data = []


# 遍历每篇文章
for article in articles["PubmedArticle"]:
    title = article["MedlineCitation"]["Article"]["ArticleTitle"]

    # 检查 'Abstract' 是否存在
    if "Abstract" in article["MedlineCitation"]["Article"]:
        abstract_parts = article["MedlineCitation"]["Article"]["Abstract"]["AbstractText"]
        # AbstractText 可能是一个字符串，也可能是一个字符串列表
        if isinstance(abstract_parts, list):
            abstract = " ".join(abstract_parts)
        elif abstract_parts is None:
            abstract = "No Abstract"
        else:
            abstract = abstract_parts
    else:
        abstract = "No Abstract"

    authors = []
    for au in article["MedlineCitation"]["Article"]["AuthorList"]:
        # 检查 'LastName' 和 'Initials' 是否存在
        if 'LastName' in au and 'Initials' in au:
            authors.append(au["LastName"] + " " + au["Initials"])

    journal = article["MedlineCitation"]["Article"]["Journal"]["Title"]
    pub_date = article["MedlineCitation"]["Article"]["Journal"]["JournalIssue"]["PubDate"]

    # 使用googletrans将摘要翻译为中文，前提是摘要不为空
    if abstract != "No Abstract":
        translator = Translator()
        try:
            translation = translator.translate(abstract, dest='zh-cn')
            abstract_cn = translation.text
        except Exception as e:
            print(f"An error occurred while translating the abstract: {abstract}")
            print(f"Error: {e}")
            abstract_cn = "Translation failed"
    else:
        abstract_cn = "No Abstract"

    # 将文章信息添加到数据列表中
    article_data.append([title, abstract, authors, journal, pub_date, abstract_cn])


# 将数据列表转换为pandas DataFrame
df = pd.DataFrame(article_data, columns=["Title", "Abstract", "Authors", "Journal", "Publication Date", "Abstract in Chinese"])

# 创建一个新的Word文档
doc = Document()

# 将每篇文章的信息添加到Word文档中
for index, row in df.iterrows():
    doc.add_heading('Title: ', level=1)
    doc.add_paragraph(row['Title'])
    doc.add_heading('Abstract: ', level=2)
    doc.add_paragraph(row['Abstract'])
    doc.add_heading('Authors: ', level=2)
    doc.add_paragraph(', '.join(row['Authors']))
    doc.add_heading('Journal: ', level=2)
    doc.add_paragraph(row['Journal'])
    doc.add_heading('Publication Date: ', level=2)
    doc.add_paragraph(str(row['Publication Date']))
    doc.add_heading('Abstract in Chinese: ', level=2)
    doc.add_paragraph(row['Abstract in Chinese'])
    doc.add_paragraph("\n\n")

# 保存Word文档
doc.save('embryo_assisted_hatching_articles.docx')


An error occurred while translating the abstract: What is the clinical value of Day 7 blastocysts? Ending embryo culture at 144 hours post-insemination (h.p.i.; i.e. 6 days) would involve 7.3% and 4.4% relative reductions in the number of patients obtaining euploid blastocysts and live birth(s) (LBs), respectively. Many studies showed that Day 7 blastocysts are clinically valuable, although less euploid and less competent than faster-growing embryos. Nevertheless, a large variability exists in: (i) the definition of 'Day 7'; (ii) the criteria to culture embryos to Day 7; (iii) the clinical setting; (iv) the local regulation; and/or (v) the culture strategies and incubators. Here, we aimed to iron out these differences and portray Day 7 blastocysts with the lowest possible risk of bias. To this end, we have also adopted an artificial intelligence (AI)-powered software to automatize developmental timings annotations and standardize embryo morphological assessment. Observational study inc